<a href="https://colab.research.google.com/github/sahil3Vedi/Django-Recommender/blob/master/Collaborative_Filtering_Movie_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# MOVIE RECOMMENDER USING COLLABORATIVE FILTERING

In [0]:
from google.colab import drive
import pandas as pd
import os
drive.mount('/content/drive')
from IPython.display import HTML, display
from numpy.random import randint

In [0]:
def progress(value, max):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [0]:
DATA_DIR = 'drive/My Drive/movielens'
moviescsv = "movie.csv"
creditscsv = "rating.csv"
MOV_PATH = os.path.join(DATA_DIR, moviescsv)
REV_PATH = os.path.join(DATA_DIR,creditscsv)
movdata = pd.read_csv(MOV_PATH)
revdata = pd.read_csv(REV_PATH)

In [0]:
list_of_movids = list(movdata["movieId"])
list_of_userIds = []
#Creating list of unique userIds:
current_user_Id = 0
for each_entry in revdata["userId"]:
  present_entry = each_entry
  if present_entry != current_user_Id:
    list_of_userIds.append(present_entry)
    current_user_Id = present_entry
user_row = list(revdata["userId"])

In [0]:
def normaliseArray(anyArray):
  min_list = min(anyArray)
  max_list = max(anyArray)
  temp = []
  for eachelem in anyArray:
    tempelem = round(-1 + (eachelem - min_list)*2/(max_list - min_list),2)
    temp.append(tempelem)
  return temp

dataframeColumns = ['userId']
for each_Id in list_of_movids:
  dataframeColumns.append(str(each_Id))

# export Pivot Matrix for users startIndex to endIndex-1 inclusive
def exportPivotMatrix(startIndex,endIndex):
  pivotMatrix = []
  dataframeIndices = []
  for every in range(startIndex, endIndex):
    dataframeIndices.append(str(every))
  pivotDf = pd.DataFrame(index=dataframeIndices,columns=dataframeColumns)
  print("Creating Cosine Matrix for users " + str(startIndex) + " to " + str(endIndex-1) + " inclusive..")
  print("Total Elements: " + str(endIndex-startIndex) + " x 27278 = " + str((endIndex-startIndex)*27278) + " elements approx..")
  #get first index of the first user
  out = display(progress(0, startIndex-endIndex), display_id=True)
  userId = startIndex
  dfCounter = 0
  while userId<endIndex:
    #obtaining user rated list
    userRatedList = []
    userRatings = []
    currentIndex = user_row.index(userId)
    nextIndex = user_row.index(userId+1)
    for revCounter in range(currentIndex,nextIndex):
      userRatedList.append(revdata["movieId"][revCounter])
      userRatings.append(revdata["rating"][revCounter])
    userRatings = normaliseArray(userRatings)
    #obtaining user rating list
    userRatingList = []
    ratingCounter = 0
    for movieIndex in list_of_movids:
      if movieIndex in userRatedList:
        userRatingList.append(userRatings[ratingCounter])
        ratingCounter += 1
      else:
        userRatingList.append(0)
    pivotMatrix.append(userRatingList)
    pivotDf.iloc[dfCounter] = [userId] + userRatingList
    out.update(progress(userId+1-startIndex,endIndex-startIndex))
    userId+=1
    dfCounter+=1
  return pivotDf
    
    
    
      

In [52]:
pivot = exportPivotMatrix(1,11)
print(pivot)

Creating Cosine Matrix for users 1 to 10 inclusive..
Total Elements: 10 x 27278 = 272780 elements approx..


   userId    1     2     3  4  5  ... 131252 131254 131256 131258 131260 131262
1       1    0  -0.5     0  0  0  ...      0      0      0      0      0      0
2       2    0     0  0.33  0  0  ...      0      0      0      0      0      0
3       3  0.5     0     0  0  0  ...      0      0      0      0      0      0
4       4    0     0     0  0  0  ...      0      0      0      0      0      0
5       5    0 -0.33     0  0  0  ...      0      0      0      0      0      0
6       6    1     0     0  0  0  ...      0      0      0      0      0      0
7       7    0     0     0  0  0  ...      0      0      0      0      0      0
8       8  0.5     0     1  0  0  ...      0      0      0      0      0      0
9       9    0     0     0  0  0  ...      0      0      0      0      0      0
10     10    0     0     0  0  0  ...      0      0      0      0      0      0

[10 rows x 27279 columns]
